## Resources
- Tiktokenizer
- Excalidraw
- Andrej Karpathy (Youtube) https://www.youtube.com/watch?v=7xTGNNLPyMI

### My Notes
- In this Notebook we're using Facebook's Open Pretrained Transformer (OPT)
- The `tokenizer` object loads the vocabulary required to interact with the model and converts the sample input (`inp` variable) to token IDs and attention mask
- The **attention mask** is a vector designed to help ignore specific tokens. (All 1's in this example, meaning we're not omitting anything)
- We use `print(OPT_model.model)` to inspect the model's architecture. You'll notice `OPTDecoder` in the output, referring to the **Decoder** Only model
- `return_tensors="pt"` in our tokenizer() instantiation means we want our tokens returned as PyTorch Tensors, not NumPy arrays (default)
- Use `nvidia-smi` to check your VRAM usage

In [3]:
!pip install transformers accelerate bitsandbytes

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

#import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


model_name = "facebook/opt-1.3b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define quantization config -- This enables efficient utilization on consumer GPUs
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

### Load the model

Note that the `.to()` method is applied to both the Model and the Tokenizer

In [6]:
torch.cuda.empty_cache()

OPT_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, low_cpu_mem_usage=True)
OPT_model.to(device) # Model is on the GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer( inp, return_tensors="pt" ).to(device) # Tokenizer is on the GPU

print( inp_tokenized['input_ids'].size() )
print( inp_tokenized )

torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [8]:
print( OPT_model.model )

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTSdpaAttention(
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwi

In [9]:
embedded_input = OPT_model.model.decoder.embed_tokens(inp_tokenized['input_ids'])
print( "Layer:\t", OPT_model.model.decoder.embed_tokens )
print( "Size:\t", embedded_input.size() )
print( "Output:\t", embedded_input )

Layer:	 Embedding(50272, 2048, padding_idx=1)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)
